In [3]:
import requests

url = 'http://localhost:5000/api/get_stories'
data = {
    'cats': 'ay.sports.nba, ay.sports.nfl',
    'word_count': '500',
    'datetime': 'NOW-1DAYS/DAY',
    'location': 'Q65',
    'writing_style': 'funny',
    'total': '5',
    'token': '344a7e67c939a28e3a250d57e3b63cf7'
}

response = requests.post(url, json=data)
response

<Response [200]>

In [1]:
data = {'cats': 'ay.sports.nba, ay.sports.nfl', 'word_count': '500', 'datetime': 'NOW-1DAYS/DAY', 'location': 'Q65', 'writing_style': 'funny', 'total': '5', 'token': '344a7e67c939a28e3a250d57e3b63cf7'}
data

{'cats': 'ay.sports.nba, ay.sports.nfl',
 'word_count': '500',
 'datetime': 'NOW-1DAYS/DAY',
 'location': 'Q65',
 'writing_style': 'funny',
 'total': '5',
 'token': '344a7e67c939a28e3a250d57e3b63cf7'}

In [2]:
# url = 'https://api.aylien.com/news/stories?
# aql=categories:({{taxonomy:aylien AND id:ay.biz.crime}} OR {{taxonomy:aylien AND id:ay.lifesoc.crime}}) 
# AND entities:({{id:Q65 AND overall_prominence:>=0.65}} OR {{id:Q30 AND overall_prominence:>=0.65}}) 
# AND sentiment.title.polarity:(negative neutral positive)
# &cursor=*&published_at.end=NOW&published_at.start=NOW-1DAYS/DAY'
categories = data['cats'].split(', ')
cat_search_string = " OR ".join([f"{{{{taxonomy: aylien AND id:{one_cat}}}}}" for one_cat in categories])
cat_search_string

'{{taxonomy: aylien AND id:ay.sports.nba}} OR {{taxonomy: aylien AND id:ay.sports.nfl}}'

In [3]:
opts = {
    'published_at_start': data['datetime'],
    'published_at_end': 'NOW',
    'language': ['en'],
    'per_page': int(data['total']),
    'source_locations_country': ['US'],
    'categories_taxonomy': 'aylien',
    'aql': f'categories: ({cat_search_string}) AND entities:({{{{id:{data["location"]} AND overall_prominence:>=0.65}}}})',
    'not_entities_id' : ['N25240567620063168378490754444048642724'], #exclude sponsored
    'sort_by': 'relevance'
}
opts

{'published_at_start': 'NOW-1DAYS/DAY',
 'published_at_end': 'NOW',
 'language': ['en'],
 'per_page': 5,
 'source_locations_country': ['US'],
 'categories_taxonomy': 'aylien',
 'aql': 'categories: ({{taxonomy: aylien AND id:ay.sports.nba}} OR {{taxonomy: aylien AND id:ay.sports.nfl}}) AND entities:({{id:Q65 AND overall_prominence:>=0.65}})',
 'not_entities_id': ['N25240567620063168378490754444048642724'],
 'sort_by': 'relevance'}

In [36]:
default_opts = {
    'published_at_start': 'NOW-2DAYS',
    'published_at_end': 'NOW',
    'language': ['en'],
    'per_page': 5,
    'source_locations_country': ['US'],
    'categories_taxonomy': 'aylien',
    'aql': 'categories: {{taxonomy: aylien AND id:ay.sports.nba}}',
    'not_entities_id' : ['N25240567620063168378490754444048642724'], #exclude sponsored
    'sort_by': 'relevance'
}

In [9]:
import os
from dotenv import load_dotenv
from __future__ import print_function
import time
import aylien_news_api
from aylien_news_api.rest import ApiException
from pprint import pprint

load_dotenv()
API_ID = os.getenv("AYLIEN_API_ID")
API_KEY = os.getenv("AYLIEN_API_KEY")

configuration = aylien_news_api.Configuration()
configuration.api_key['X-AYLIEN-NewsAPI-Application-ID'] = API_ID
configuration.api_key['X-AYLIEN-NewsAPI-Application-Key'] = API_KEY

client = aylien_news_api.ApiClient(configuration)
api_instance = aylien_news_api.DefaultApi(client)

In [5]:
try:
    api_response = api_instance.list_stories(**opts)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling DefaultApi->list_stories: %s\n" % e)

{'next_page_cursor': 'MTUuMDQ2MjU0LDUzNzcxODI5NjQ=',
 'published_at_end': datetime.datetime(2023, 3, 8, 18, 58, 12, 496012, tzinfo=tzutc()),
 'published_at_start': datetime.datetime(2023, 3, 7, 0, 0, tzinfo=tzutc()),
 'stories': [{'author': {'avatar_url': None, 'id': 3040616, 'name': ''},
              'body': 'The Los Angeles Lakers beat the Memphis Grizzlies on '
                      'Tuesday night.  Continue reading\n'
                      ' There are several quarterback-needy teams in the NFL, '
                      'but no reports have surfaced so far of those that will '
                      'pursue Lamar Jackson.\n'
                      '  The Panthers have joined the list of those that '
                      "reportedly won't.\n"
                      '  Joe Person of TheAthletic.com reports that General '
                      'Manager Scott Fitterer will discuss and has discussed '
                      "Jackson “because that's what he does, but the Panthers "
         

In [7]:
import json
# import pandas as pd

# type(api_response)     
json_response = [
    {
        'id': story.id,
        'body': story.body, 
        'author': story.author.to_dict(),
        'links': story.links.to_dict(),
        'media': [image.url for image in story.media],
        'published_at': str(story.published_at),
        'sentiment': story.sentiment.to_dict(),
        'summary': story.summary.to_dict(),
        'title': story.title,
        'words_count': story.words_count,
        'categories': [category.label for category in story.categories],
        'characters_count': story.characters_count,
        'sentences_count': story.sentences_count,
        'source': story.source.name
    } 
    for story in api_response.stories]

# class CustomEncoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, StoryLinks):
#             return obj.to_dict()
#         return super().default(obj)
# json_response = {k: v for k, v in api_response.stories.items() if k in ('author', 'body')}
#     print(story.entities)
#         story.published_at = str(story.published_at)
#     print(story)
#         del story.entities
# response_dict = api_response.to_dict()
response_str = json.dumps(json_response)
# response_str
# response_dict['stories']
# api_response.stories
response_str
# json_response

'[{"id": 5376796638, "body": "The Los Angeles Lakers beat the Memphis Grizzlies on Tuesday night.  Continue reading\\n There are several quarterback-needy teams in the NFL, but no reports have surfaced so far of those that will pursue Lamar Jackson.\\n  The Panthers have joined the list of those that reportedly won\'t.\\n  Joe Person of TheAthletic.com reports that General Manager Scott Fitterer will discuss and has discussed Jackson \\u201cbecause that\'s what he does, but the Panthers are not expected to be in the Jackson market.\\u201d\\n  The Ravens tagged the quarterback at the non-exclusive level, allowing him to negotiate a contract with other teams. Baltimore will have the opportunity to match an offer sheet or receive two first-round draft picks if it doesn\'t.\\n  Not long after news of Jackson\'s tag came reports that the Falcons will not have interest in Jackson.\\n  The Panthers did have interest in Derek Carr , meeting with him before he signed with the Saints. Coach Fran

In [ ]:
# json = json_response.to_json()
# json

In [10]:
original_opts = {
    'id': [5379348412],
    'per_page': 1,
    'sort_by': 'relevance'
}

In [11]:
try:
    api_response = api_instance.list_stories(**original_opts)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling DefaultApi->list_stories: %s\n" % e)

{'next_page_cursor': None,
 'published_at_end': None,
 'published_at_start': None,
 'stories': [{'author': {'avatar_url': None,
                         'id': 1159961847,
                         'name': 'Emma Reynolds Wed'},
              'body': 'Emma Reynolds  The tony guard-gated neighborhood of '
                      'Hidden Hills in Los Angeles is known for its scores of '
                      'celebrities, athletes, musicians and influencers.\n'
                      "One celeb-owned home that's bouncing back on the "
                      'market? 24350 Bridle Trail, otherwise known as the home '
                      'of Australian NBA All-Star and Brooklyn Nets player Ben '
                      'Simmons.\n'
                      '  Listed for $19.5 million, the modern farmhouse '
                      'retreat sits at the end of a private cul-de-sac with '
                      'lush landscaping, ideal for the privacy-seeking buyer. '
                      'It was designed

In [15]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

def get_summary(article, count):
    LANGUAGE = "english"
    SENTENCES_COUNT = count

    if __name__ == "__main__":
        parser = PlaintextParser.from_string(article, Tokenizer(LANGUAGE))
        stemmer = Stemmer(LANGUAGE)

        summarizer = Summarizer(stemmer)
        summarizer.stop_words = get_stop_words(LANGUAGE)

        summary = " ".join(str(sentence) for sentence in summarizer(parser.document, SENTENCES_COUNT))
        return summary
    
def convert_story(story):
    converted = {
        'text': get_summary(story.body, 8), 
        'entrities': format_entities(story.entities), 
        'links': story.links.permalink,
        'authors': story.author.name,
        'thumbnails':  story.media[0].url if len(story.media) > 0 else None
    }
    
    converted['text'] = clean_text_author(story.author.name, converted['text'])
    converted['entrities'] = clean_ents_author(story.author.name, converted['entrities'])
    return converted

def format_entities(ents):
    ents_output = [
        (entity.body.surface_forms[0].text, entity.types[0], entity.overall_frequency)
        for entity in ents
        if len(entity.body.surface_forms) > 0 and len(entity.types) > 0 and isinstance(entity.overall_frequency, (int, float))
    ]
    
    if len(ents_output) > 30:
        ents_output = [item[:2] for item in ents_output if item[2] > 1]
    else:
        ents_output = [item[:2] for item in ents_output]
        
    return ents_output

def clean_ents_author(author, ents):
    for entity in ents:
        if author in entity:
            ents.remove(entity)    
    return ents

def clean_text_author(author, text):
    while author in text:
        text = text.replace(author, "")
    return text

c1 = convert_story(api_response.stories[0])
c1

C:\Users\beaconproduction\sites\local\python\heywire\openai\venv\Lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (5.1.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


{'text': "Emma Reynolds  The tony guard-gated neighborhood of Hidden Hills in Los Angeles is known for its scores of celebrities, athletes, musicians and influencers. “The contemporary kitchen, perfect for relaxed entertaining, seamlessly flows into the outdoor patio complete with a cozy fireplace, where you can enjoy stunning views of the 65-foot pool. The heart of the property is its great room with double-height vaulted ceilings, panoramic windows and direct backyard access, as well as sleek Pietra marble flooring. Head through the great room's the automatic Fleetwood sliding glass pocket doors for immediate access to the backyard, where you'll find an alfresco dining table and a fireplace. Across the pool, there's a luxe cabana with a bevy of amemtities, including a a covered sitting area, a billiards table, a bathroom, a built-in-barbecue and a video wall. The upstairs, meanwhile, is accessed through the staircase in the family room, which has a live Zen garden with a bonsai tree 

In [5]:
import sys
sys.path.append('../application/api/')
import formats
import aylien

import importlib
importlib.reload(aylien)

# original_story = aylien.get_one_story(5379348412)
# relatives = aylien.get_relatives(5379348412)
full = aylien.full_story(5379348412)
full_combined = formats.combine_story(full)
full_combined
# relatives
# original_story

{'text': 'Emma Reynolds  The tony guard-gated neighborhood of Hidden Hills in Los Angeles is known for its scores of celebrities, athletes, musicians and influencers. Listed for $19.5 million, the modern farmhouse retreat sits at the end of a private cul-de-sac with lush landscaping, ideal for the privacy-seeking buyer. “The contemporary kitchen, perfect for relaxed entertaining, seamlessly flows into the outdoor patio complete with a cozy fireplace, where you can enjoy stunning views of the 65-foot pool. The heart of the property is its great room with double-height vaulted ceilings, panoramic windows and direct backyard access, as well as sleek Pietra marble flooring. Head through the great room\'s the automatic Fleetwood sliding glass pocket doors for immediate access to the backyard, where you\'ll find an alfresco dining table and a fireplace. Across the pool, there\'s a luxe cabana with a bevy of amemtities, including a a covered sitting area, a billiards table, a bathroom, a buil

In [7]:
import sys
sys.path.append('../application/api/')
import gpt
# import aylien

import importlib
importlib.reload(gpt)

gs = gpt.generate_story(
    full_combined['links'], 
    full_combined['entities'], 
    full_combined['text']
)
gs

'"Rare Gem" in Martinsville: A Spectacular Find in Henry County\n\nWhen the 7 Bedroom Home in Martinsville was listed on the market, it became an instant hit among the residents of the Tri-Valley School District. The huge property boasted of breathtaking views and state-of-the-art amenities, and was the talk of the town. The property was recently featured in the Robb Report and earned the Michelin rating for its sheer elegance and attention to detail.\n\nThe home belonged to the Salgado family, who had built the house from scratch. Rachel Salgado, the matriarch of the family, had an impeccable taste and an eye for detail. She had ensured that every aspect of the house exuded luxury and comfort. The family had decided to sell the property and move to a French chateau built by renowned architect Bassett.\n\nThe Salgado family had enlisted the help of Emma Reynolds, a popular real estate agent from Williams & Williams, who had previously represented clients in the Beverly Hills Estates an

In [55]:
len(relatives)

10

In [44]:
relatives = aylien.get_relatives(5379348412)
relatives

Opts:  {'published_at_end': 'NOW', 'language': ['en'], 'source_locations_country': ['US'], 'categories_taxonomy': 'aylien', 'not_entities_id': ['N25240567620063168378490754444048642724'], 'sort_by': 'relevance', 'id': [5379348412], 'per_page': 2, 'story_id': 5379348412, 'not_id': [5379348412]}


RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.

In [23]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

def get_summary(article, count):
    print('im here')
    LANGUAGE = "english"
    SENTENCES_COUNT = count

    if __name__ == "__main__":
        parser = PlaintextParser.from_string(article, Tokenizer(LANGUAGE))
        stemmer = Stemmer(LANGUAGE)

        summarizer = Summarizer(stemmer)
        summarizer.stop_words = get_stop_words(LANGUAGE)

        summary = " ".join(str(sentence) for sentence in summarizer(parser.document, SENTENCES_COUNT))
        return summary
sum1 = get_summary(api_response.stories[0].body,8)
sum1

im here


"Emma Reynolds  The tony guard-gated neighborhood of Hidden Hills in Los Angeles is known for its scores of celebrities, athletes, musicians and influencers. “The contemporary kitchen, perfect for relaxed entertaining, seamlessly flows into the outdoor patio complete with a cozy fireplace, where you can enjoy stunning views of the 65-foot pool. The heart of the property is its great room with double-height vaulted ceilings, panoramic windows and direct backyard access, as well as sleek Pietra marble flooring. Head through the great room's the automatic Fleetwood sliding glass pocket doors for immediate access to the backyard, where you'll find an alfresco dining table and a fireplace. Across the pool, there's a luxe cabana with a bevy of amemtities, including a a covered sitting area, a billiards table, a bathroom, a built-in-barbecue and a video wall. The upstairs, meanwhile, is accessed through the staircase in the family room, which has a live Zen garden with a bonsai tree underneat